In [1]:
require(Matrix)
require(tibble)
utilities_path="../correlation_network_analysis_functions.r"
source(utilities_path)

Loading required package: Matrix
Loading required package: tibble
Loading required package: mgcv
Loading required package: nlme
This is mgcv 1.8-16. For overview type 'help("mgcv-package")'.
Loading required package: MASS
Loading required package: ggplot2
Loading required package: dplyr

Attaching package: ‘dplyr’

The following object is masked from ‘package:MASS’:

    select

The following object is masked from ‘package:nlme’:

    collapse

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: reshape2
Loading required package: RColorBrewer
Loading required package: colorRamps
Loading required package: tidyr

Attaching package: ‘tidyr’

The following object is masked from ‘package:reshape2’:

    smiths

The following object is masked from ‘package:Matrix’:

    expand

Loading required package: colorspace
Loading required package: igraph

Att

In [2]:
basepath="./"
baseinfodir="./"
analysisDir=paste(basepath,"analysis",sep="/")
datafilename="contact.corr"
contactmapname="contact.contact"
coordfilename="carma.average.pdb"
resinfofilename="resinfo.cpptraj.dat"
atominfofilename="atominfo.cpptraj.dat"
system_list=c("wt","R206H","Q207E","Q207D")
window_list=c(0:4)

In [ ]:
#only need to do this once, if restarting later, proceed to the step
# READ IN CORRELATION DATA FROM JOINT DATA TABLE
#load the contact - correlation maps generated by carma into a single data frame
#and apply some residue naming information
data_list <- load_windowed_correlation_data(basepath=basepath,baseinfodir=baseinfodir,
                                            datafilename=datafilename,
                                            contactmapname=contactmapname,
                                            resinfofilename=resinfofilename,
                                            atominfofilename=atominfofilename,
                                            coordfilename=coordfilename,
                                            system_list=system_list,window_list=window_list,
                                            skip_corr_header_rows=1,
                                            datafiles_as_arrays=TRUE,
                                            contactfiles_as_arrays=TRUE)
windowed_corr_data <- data_list[["windowed_corr_data"]]
windowed_contact_map <- data_list[["windowed_contact_data"]]
resData <- data_list[["res_data"]]
contact_corr <- gen_contact_corr_summary(windowed_corr_data,windowed_contact_map,
                                             verbose=TRUE)
contact_corr %>% head
data_list %>% glimpse

In [ ]:
# SAVE DATA TO JOINT TABLE
write.table(x=data_list$windowed_corr_data,
            file=paste(analysisDir,"/joint_windowed_correlation_data_table.dat",sep=""),
            quote=FALSE,row.names=FALSE)
write.table(x=contact_corr,
            file=paste(analysisDir,"/joint_contact_averaged_correlation_data_table.dat",sep=""),
            quote=FALSE,row.names=FALSE)
write.table(x=data_list$res_data,
            file=paste(analysisDir,"/joint_res_data_table.dat",sep=""),
            quote=FALSE,row.names=FALSE)

In [3]:
# READ IN CORRELATION DATA FROM JOINT DATA TABLE
equil_windowed_corr_data=read.table(
    paste(analysisDir,"/joint_windowed_correlation_data_table.dat",sep=""),header=TRUE) %>%
    filter(WINDOW>0) %>%
    mutate(CONTACT=1.0*(abs(CORR)>0))
print("Windowed Correlation Data")
equil_windowed_corr_data %>% head

#Read in residue information data
res_data <- read.table("joint_res_data_table.dat",header=TRUE)
print("Residue Information")
res_data %>% head

[1] "Windowed Correlation Data"


SYSTEM,WINDOW,X,Y,X_RES,Y_RES,X_RESNAME,Y_RESNAME,X_Residue,Y_Residue,CORR,CONTACT
wt,1,1,1,172,172,THR,THR,THR_172,THR_172,0.00000,0
wt,1,2,1,173,172,THR,THR,THR_173,THR_172,0.00000,0
wt,1,3,1,174,172,ASN,THR,ASN_174,THR_172,0.86239,1
wt,1,4,1,175,172,VAL,THR,VAL_175,THR_172,0.00000,0
wt,1,5,1,176,172,GLY,THR,GLY_176,THR_172,0.00000,0
wt,1,6,1,177,172,ASP,THR,ASP_177,THR_172,0.00000,0


[1] "Residue Information"


SYSTEM,ResNum,RESID,RESNAME,Residue,rX,rY,rZ
wt,1,172,THR,THR_172,30.788,82.529,60.337
wt,2,173,THR,THR_173,32.126,80.358,60.854
wt,3,174,ASN,ASN_174,31.539,77.784,60.611
wt,4,175,VAL,VAL_175,32.069,75.231,62.085
wt,5,176,GLY,GLY_176,32.769,72.207,62.834
wt,6,177,ASP,ASP_177,35.012,71.432,60.613


In [11]:
# COMPUTE WINDOWED FLOW BETWEENNESS DATA
# This only needs to be run once, unless you want to change the number of windows
# or add new networks.
# If you already did this and are picking up where you left off, proceed to
# the cell for 'LOAD WINDOWED CORRELATION AND FLOW BETWEENNESS DATA FROM FILE'
system_list=c("wt","R206H","Q207E","Q207D")
window_list=c(1:4) #Window 0 was discarded as addtional 'equilibration' time
#Define named sets of source and target nodes for flow betweenness analysis
#Most of this is for built in book-keeping purposes to make later statistical
#analyses easier
sourcesets_list=list()
targetsets_list=list()
sources=c("FKBP")
targets=c("res375")
for(system in system_list){
    sourcesets_list[[system]]=sources
    targetsets_list[[system]]=targets
}

sourcenode_list=list()
sourcenode_list[["FKBP"]]=c(353,363,364,365,368,369,371,373,374,375,376,379,380,381,
                            382,383,384,386,408,409,411,412,413,414,415,416,417,418,420,424,426,434)
targetnode_list=list()
targetnode_list[["res375"]]=c(204)

windowed_network_data <- do_windowed_betweenness_analysis(
    windowed_corr_data=equil_windowed_corr_data,resData=res_data,
    sourceSetsList=sourcesets_list,targetSetsList=targetsets_list,
    sourceNodeList=sourcenode_list,targetNodeList=targetnode_list,
    nRes=max(c(windowed_corr_data$X,windowed_corr_data$Y)),
    windows=window_list,
    systems=system_list,
    includeGraphList=FALSE,verbose=TRUE,useMessage=TRUE,verboseLevel=1) %>%
    rename(BTW=btw)

windowed_network_data%>%head

------------------------------
Working on  wt  system
-Working on window 1
--source sets:FKBP
--target sets:res375
--Working on FKBP_res375 network


[1] "Generating contact correlation graph"
[1] "Generating Current-Flow-Betweenness data"
[1] "Constructing plot graph from contact graph and betweenness data"
[1] "Done!"


-Working on window 2
--source sets:FKBP
--target sets:res375
--Working on FKBP_res375 network


[1] "Generating contact correlation graph"
[1] "Generating Current-Flow-Betweenness data"
[1] "Constructing plot graph from contact graph and betweenness data"
[1] "Done!"


-Working on window 3
--source sets:FKBP
--target sets:res375
--Working on FKBP_res375 network


...

-Working on window 3
--source sets:FKBP
--target sets:res375
--Working on FKBP_res375 network


[1] "Generating contact correlation graph"
[1] "Generating Current-Flow-Betweenness data"
[1] "Constructing plot graph from contact graph and betweenness data"
[1] "Done!"


-Working on window 4
--source sets:FKBP
--target sets:res375
--Working on FKBP_res375 network


[1] "Generating contact correlation graph"
[1] "Generating Current-Flow-Betweenness data"
[1] "Constructing plot graph from contact graph and betweenness data"
[1] "Done!"


SYSTEM,WINDOW,NETWORK,X,Y,X_Y,CORR,BTW
wt,1,FKBP_res375,1,3,1_3,0.86239,6.233292e-15
wt,1,FKBP_res375,2,4,2_4,0.86893,7.842461e-15
wt,1,FKBP_res375,3,5,3_5,0.69314,1.252439e-14
wt,1,FKBP_res375,5,7,5_7,0.47262,1.808535e-14
wt,1,FKBP_res375,7,9,7_9,0.45404,2.174553e-14
wt,1,FKBP_res375,8,10,8_10,0.72660,2.283932e-04


In [13]:
#Save windowed correlation and flow betweenness data
#Be sure to use unique names if you want to save data for different sets of
#networks, windows, or systems, or pass a file object in append mode.
write.table(windowed_network_data,"Joint_Windowed_Corr_Flow_Data.dat",
            row.names=FALSE,quote=FALSE)

In [3]:
#LOAD WINDOWED CORRELATION AND FLOW BETWEENNESS DATA FROM FILE
windowed_network_data <- read.table("Joint_Windowed_Corr_Flow_Data.dat",header=TRUE)
print("Windowed correlation and flow betweenness data")
windowed_network_data %>% head

res_data <- read.table("joint_res_data_table.dat",header=TRUE)
print("Residue information")
res_data %>% head

[1] "Windowed correlation and flow betweenness data"


SYSTEM,WINDOW,NETWORK,X,Y,X_Y,CORR,BTW
wt,1,FKBP_res375,1,3,1_3,0.86239,6.233292e-15
wt,1,FKBP_res375,2,4,2_4,0.86893,7.842461e-15
wt,1,FKBP_res375,3,5,3_5,0.69314,1.252439e-14
wt,1,FKBP_res375,5,7,5_7,0.47262,1.808535e-14
wt,1,FKBP_res375,7,9,7_9,0.45404,2.174553e-14
wt,1,FKBP_res375,8,10,8_10,0.72660,2.283932e-04


[1] "Residue information"


SYSTEM,ResNum,RESID,RESNAME,Residue,rX,rY,rZ
wt,1,172,THR,THR_172,30.788,82.529,60.337
wt,2,173,THR,THR_173,32.126,80.358,60.854
wt,3,174,ASN,ASN_174,31.539,77.784,60.611
wt,4,175,VAL,VAL_175,32.069,75.231,62.085
wt,5,176,GLY,GLY_176,32.769,72.207,62.834
wt,6,177,ASP,ASP_177,35.012,71.432,60.613


In [13]:
windowedNetworkMatricesDir=paste(analysisDir,"windowed_network_matrices",sep="/")
systems=windowed_network_data$SYSTEM%>%unique
networks=windowed_network_data$NETWORK%>%unique
windows=windowed_network_data$WINDOW%>%unique
long_windowed_network_data <- windowed_network_data %>%
    melt(id.vars = c("SYSTEM","WINDOW","NETWORK","X","Y","X_Y"),
         variable.name="WEIGHT_TYPE",
         value.name="VALUE")
long_windowed_network_data%>%head

weightTypes=long_windowed_network_data$WEIGHT_TYPE %>% unique

SYSTEM,WINDOW,NETWORK,X,Y,X_Y,WEIGHT_TYPE,VALUE
wt,1,FKBP_res375,1,3,1_3,CORR,0.86239
wt,1,FKBP_res375,2,4,2_4,CORR,0.86893
wt,1,FKBP_res375,3,5,3_5,CORR,0.69314
wt,1,FKBP_res375,5,7,5_7,CORR,0.47262
wt,1,FKBP_res375,7,9,7_9,CORR,0.45404
wt,1,FKBP_res375,8,10,8_10,CORR,0.72660


In [14]:
nodecount=435
message("Writting windowed network matrices for use in suboptimal path analysis")
for (system in systems) {
    message(paste("Working on",system,"system"))
    for (window in windows) {
        message(paste("-Working on window",window))
        for (network in networks) {
            message(paste("--Working on",network,"sub-network"))
            for (weightType in weightTypes) {
                message(paste("---Working on",weightType,"weights"))
                message("----computing weights from value data")
                dataSubset<-long_windowed_network_data%>%
                    filter(SYSTEM==system,WINDOW==window,
                           NETWORK==network,WEIGHT_TYPE==weightType,
                           abs(VALUE)>0) %>%
                    mutate(WEIGHT=-log(abs(VALUE)))
                valueMatrixFileName=paste(paste(system,network,sep="_"),
                                          paste("window",window,sep="_"),
                                          paste(weightType,"value",sep="_"),
                                          "dat",sep=".")
                weightMatrixFileName=paste(paste(system,network,sep="_"),
                                           paste("window",window,sep="_"),
                                           paste(weightType,"weight",sep="_"),
                                           "dat",sep=".")
                valueMatrixFilePath=paste(windowedNetworkMatricesDir,
                                          valueMatrixFileName,sep="/")
                weightMatrixFilePath=paste(windowedNetworkMatricesDir,
                                           weightMatrixFileName,sep="/")

                value_data<-dataSubset%>%select(X,Y,VALUE)
                weight_data<-dataSubset%>%select(X,Y,WEIGHT)
                
                message(paste("----writting value matrix to:",
                              valueMatrixFilePath))
                write_networkDataTable_to_networkView_matrix(
                    outputFileName=valueMatrixFilePath,networkData=value_data,
                    toColumn="Y",fromColumn="X",valColumn="VALUE",
                    nodeCount=nodecount,makeSymmetric=TRUE)

                message(paste("----writting weight matrix to:",
                        weightMatrixFilePath))
                write_networkDataTable_to_networkView_matrix(
                    outputFileName=weightMatrixFilePath,networkData=weight_data,
                    toColumn="Y",fromColumn="X",valColumn="WEIGHT",
                    nodeCount=nodecount,makeSymmetric=TRUE)
            }
        }
    }
}
message("Done")

Working on wt system
-Working on window 1
--Working on FKBP_res375 sub-network
---Working on CORR weights
----computing weights from value data
Warning message:
“closing unused connection 4 (.//analysis/wt_FKBP_res375.window_1.BTW_value.dat)”----writting value matrix to: .//analysis/windowed_network_matrices/wt_FKBP_res375.window_1.CORR_value.dat
----writting weight matrix to: .//analysis/windowed_network_matrices/wt_FKBP_res375.window_1.CORR_weight.dat
---Working on BTW weights
----computing weights from value data
----writting value matrix to: .//analysis/windowed_network_matrices/wt_FKBP_res375.window_1.BTW_value.dat
----writting weight matrix to: .//analysis/windowed_network_matrices/wt_FKBP_res375.window_1.BTW_weight.dat
-Working on window 2
--Working on FKBP_res375 sub-network
---Working on CORR weights
----computing weights from value data
----writting value matrix to: .//analysis/windowed_network_matrices/wt_FKBP_res375.window_2.CORR_value.dat
----writting weight matrix to: .//

In [79]:
#compute suboptimal paths for each window of each system
outdir=paste(analysisDir,"windowed_subopt","dilationPercent_1",sep="/")
weightdir=windowedNetworkMatricesDir
suboptexe="subopt"

message(paste("output directory is:",outdir))
message(paste("network matrix directory is:",weightdir))
#carma indices start at 0, R indices start at 1
#set offset to 1 for network analysis routines done in R.
offset=-1 #inices are being fed to vmd / carma program

system_list=c("wt","R206H","Q207E","Q207D")
window_list=c(1:4) #Window 0 was discarded as addtional 'equilibration' time
#Define named sets of source and target nodes for flow betweenness analysis
#Most of this is for built in book-keeping purposes to make later statistical
#analyses easier
sourcesets_list=list()
targetsets_list=list()
sources=c("FKBP")
targets=c("res375")
for(system in system_list){
    sourcesets_list[[system]]=sources
    targetsets_list[[system]]=targets
}

sourcenode_list=list()
sourcenode_list[["FKBP"]]=c(353,363,364,365,368,369,371,373,374,375,376,379,380,381,
                            382,383,384,386,408,409,411,412,413,414,415,416,417,418,420,424,426,434)
targetnode_list=list()
targetnode_list[["res375"]]=c(204)

weightTypes<-long_windowed_network_data$WEIGHT_TYPE%>%
    unique %>%
    paste("weight",sep="_")

#these values were generated based on prelimary runs on the wt system and correspond
#to roughly 1% of the opitmal path length
dilationList=list(CORR_weight=4,BTW_weight=15)

message(paste("Using the external command '",suboptexe,"' to compute suboptimal paths"))
run_windowed_external_subopt(systems=systems,windows,
                    sourceSetsList=sourcesets_list,targetSetsList=targetsets_list,
                    sourceNodeList=sourcenode_list,targetNodeList=targetnode_list,
                    weightTypes=weightTypes,dilationList=dilationList,
                    outdir=outdir,weightdir=weightdir,suboptexe=suboptexe,
                    verbose=TRUE,verboseLevel=3)

output directory is: .//analysis/windowed_subopt/dilationPercent_1
network matrix directory is: .//analysis/windowed_network_matrices
Using the external command ' subopt ' to compute suboptimal paths


Working on wt system
-working on FKBP_res375 network
--working on window 1
---running subopt program for the 204_353 source-target pair
----weightFilePath: .//analysis/windowed_network_matrices/wt_FKBP_res375.window_1.CORR_weight.dat
----outFilePath: .//analysis/windowed_subopt/dilationPercent_1/wt_FKBP_res375.window_1.CORR_weight.204_353.subopt.out
---running subopt program for the 204_363 source-target pair
----weightFilePath: .//analysis/windowed_network_matrices/wt_FKBP_res375.window_1.CORR_weight.dat
----outFilePath: .//analysis/windowed_subopt/dilationPercent_1/wt_FKBP_res375.window_1.CORR_weight.204_363.subopt.out
---running subopt program for the 204_364 source-target pair
----weightFilePath: .//analysis/windowed_network_matrices/wt_FKBP_res375.window_1.CORR_weight.dat
----outFilePath: .//analysis/windowed_subopt/dilationPercent_1/wt_FKBP_res375.window_1.CORR_weight.204_364.subopt.out
---running subopt program for the 204_365 source-target pair
----weightFilePath: .//analysis/w